In [ ]:
# Copyright 2018 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [START gmail_quickstart]
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

CREDENTIALS_PATH = "credentials.json.env"
TOKEN_PATH = "token.json.env"

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists(TOKEN_PATH):
    creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_PATH, SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open(TOKEN_PATH, "w") as token:
        token.write(creds.to_json())


# Call the Gmail API
service = build("gmail", "v1", credentials=creds)
label_results = service.users().labels().list(userId="me").execute()
labels = label_results.get("labels", [])

print("Labels:")
print(labels)

In [ ]:
# import base64
# import email

# def get_messages(service, user_id):
#   try:
#     return service.users().messages().list(userId=user_id).execute()
#   except Exception as error:
#     print('An error occurred: %s' % error)

# def get_message(service, user_id, msg_id):
#   try:
#     return service.users().messages().get(userId=user_id, id=msg_id, format='full').execute()
#   except Exception as error:
#     print('An error occurred: %s' % error)

# def get_mime_message(service, user_id, msg_id):
#   try:
#     message = service.users().messages().get(userId=user_id, id=msg_id,
#                                              format='raw').execute()
#     print('Message snippet: %s' % message['snippet'])
#     msg_str = base64.urlsafe_b64decode(message['raw'].encode("utf-8")).decode("utf-8")
#     mime_msg = email.message_from_string(msg_str)
#     return mime_msg
#   except Exception as error:


#     print('An error occurred: %s' % error)

In [ ]:
print(messages)

In [ ]:
import base64
from bs4 import BeautifulSoup

def get_text_from_email_payload(payload):
        
    # Check if the email has multiple parts
    body = ""
    if 'parts' in payload:
        for part in payload['parts']:
            if part['mimeType'] == 'text/plain':
                body += base64.urlsafe_b64decode(part['body']['data']).decode('utf-8')
            else:
                body += get_text_from_email_payload(part)
    elif payload['mimeType'] == 'text/plain':
        body = base64.urlsafe_b64decode(payload['body']['data']).decode('utf-8')
    else:
        pass

    text_only = BeautifulSoup(body).get_text()

    return text_only

messages_list_results = service.users().messages().list(userId='me').execute()
messages = messages_list_results.get('messages', [])
print(f"total messages: {len(messages)}")

for message in messages:
    msg = service.users().messages().get(userId='me', id=message['id'], format="full").execute()
    payload = msg['payload']


    email_text = get_text_from_email_payload(payload)
    
    # print(f'Email body: {email_text}')
    if email_text == "":
        print(payload)

